# Load libraries

In [1]:
import pandas as pd

from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

import pinecone
from langchain.vectorstores import Pinecone

import os
from dotenv import load_dotenv

/Users/kieranschubert/Desktop/unigpt-assist/venv_unigpt/lib/python3.11/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


# Load dotenv

In [2]:
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_ENVIRONMENT = os.getenv('PINECONE_ENVIRONMENT')
PINECONE_INDEX = os.getenv('PINECONE_INDEX')

# Init Pinecone VectorDB

In [4]:
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENVIRONMENT,
)

if PINECONE_INDEX not in pinecone.list_indexes():
    pinecone.create_index(PINECONE_INDEX, dimension=1536, metric="cosine")

index = pinecone.Index(PINECONE_INDEX)

print(pinecone.list_indexes())
print(pinecone.describe_index(PINECONE_INDEX))
print(index.describe_index_stats())

['hackathon']
IndexDescription(name='hackathon', metric='cosine', replicas=1, dimension=1536.0, shards=1, pods=1, pod_type='starter', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')
{'dimension': 1536,
 'index_fullness': 0.00019,
 'namespaces': {'': {'vector_count': 19}},
 'total_vector_count': 19}


In [5]:
embedding_function = OpenAIEmbeddings(model="text-embedding-ada-002",
                               disallowed_special=())

vectorstore = Pinecone.from_existing_index(
            index_name=PINECONE_INDEX,
            embedding=embedding_function,
            #namespace=namespace
            )

k = 10

retriever = vectorstore.as_retriever(search_kwargs={"k": k}, return_source_documents=True)

In [ ]:
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY,
                 model="gpt-4-1106-preview",
                 temperature=0)

In [ ]:
template = """
        Vous êtes un assistant qui répond à des questions sur l'Université de Genève, basée en Suisse.
        Utilisez les éléments de contexte et l'historique du chat suivants pour répondre aux questions. 
        Votre réponse doit être liée à l'Université de Genève uniquement. Si la question ne figure pas dans le contexte ou l'historique du chat, répondez "Je suis désolé, je ne connais pas la réponse".
        Les réponses doivent être détaillées mais concises et courtes.
        Respirez profondément et travaillez étape par étape.
        
        Context: {context}

        Question: {question}
        Answer: """

prompt = PromptTemplate(input_variables=["context", "question"], template=template)

In [ ]:
qa = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=retriever,
            chain_type_kwargs={"prompt": prompt},
            return_source_documents=True
            )

In [ ]:
res = qa({"query": "J'ai envie de prendre un congé durant mes études, quelle est la durée maximum autorisée?"})

In [ ]:
res["query"]

In [ ]:
res["result"]

In [ ]:
res["source_documents"]